In [ ]:
import numpy as np
import numpy.random as nrand
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [ ]:
model_list = ['NK','RMF','Polynomial']
metric_list = ['N_max','epi','r_s','open_ratio']
N_list = [5,10,15]
error_list = [0.02,0.04,0.06,0.08]

In [ ]:
plot_dict = {N: {model: {metric: {} for metric in metric_list} for model in model_list} for N in N_list}
for N in N_list:
    for model in model_list:
        for metric in metric_list:
            with open(f'plot_data/plot_df_{model}_{N}_{metric}.pkl', 'rb') as f:
                plot_dict[N][model][metric] = pickle.load(f)

In [ ]:
para_dict = {
    5:{
        'NK':{
            'N_max':{'j':[1,8],'sd':1,'lim':[0,10],'labelpad':15},#
            'epi':{'j':[0,12],'sd':0.02,'lim':[0,0.3],'labelpad':22},#
            'r_s':{'j':[0,6],'sd':0.8,'lim':[0,10],'labelpad':13},#
            'open_ratio':{'j':[0,16],'sd':0.008,'lim':[0,0.15],'labelpad':30}
        },
        'RMF':{
            'N_max':{'j':[1,8],'sd':1,'lim':[0,10],'labelpad':15},#
            'epi':{'j':[0,12],'sd':0.02,'lim':[0,0.3],'labelpad':22},#
            'r_s':{'j':[0,8],'sd':0.8,'lim':[0,10],'labelpad':13},#
            'open_ratio':{'j':[0,16],'sd':0.008,'lim':[0,0.15],'labelpad':30}
        },
        'Polynomial':{
            'N_max':{'j':[0,4],'sd':1,'lim':[0,10],'labelpad':15}, #
            'epi':{'j':[0,8],'sd':0.01,'lim':[0,0.3],'labelpad':22},#
            'r_s':{'j':[0,8],'sd':0.2,'lim':[0,4],'labelpad':13},#
            'open_ratio':{'j':[3,18],'sd':0.008,'lim':[0,0.15],'labelpad':30}
        }
    },
    10:{
        'NK':{
            'N_max':{'j':[0,20],'sd':5,'lim':[0,110],'labelpad':20}, #
            'epi':{'j':[0,12],'sd':0.02,'lim':[0,0.2],'labelpad':30},#
            'r_s':{'j':[0,10],'sd':2,'lim':[0,20],'labelpad':13},#
            'open_ratio':{'j':[0,16],'sd':0.008,'lim':[0,0.15],'labelpad':30}
        },
        'RMF':{
            'N_max':{'j':[0,20],'sd':5,'lim':[0,110],'labelpad':20}, #
            'epi':{'j':[0,12],'sd':0.02,'lim':[0,0.2],'labelpad':30},#
            'r_s':{'j':[0,10],'sd':2,'lim':[0,20],'labelpad':13},#
            'open_ratio':{'j':[0,16],'sd':0.008,'lim':[0,0.15],'labelpad':30}
        },
        'Polynomial':{
            'N_max':{'j':[1,10],'sd':1,'lim':[0,75],'labelpad':20},#
            'epi':{'j':[0,8],'sd':0.01,'lim':[0,0.2],'labelpad':30},#
            'r_s':{'j':[0,10],'sd':0.3,'lim':[0,4],'labelpad':13},#
            'open_ratio':{'j':[8,18],'sd':0.008,'lim':[0,0.15],'labelpad':30}
        }
    },
    15:{
        'NK':{
            'N_max':{'j':[-1,9],'sd':250,'lim':[0,2300],'labelpad':40},#
            'epi':{'j':[0,12],'sd':0.02,'lim':[0,0.2],'labelpad':35},#
            'r_s':{'j':[0,13],'sd':10,'lim':[0,150],'labelpad':25},#
            'open_ratio':{'j':[0,16],'sd':0.008,'lim':[0,0.15],'labelpad':30}
        },
        'RMF':{
            'N_max':{'j':[0,18],'sd':150,'lim':[0,2300],'labelpad':40},#
            'epi':{'j':[0,12],'sd':0.02,'lim':[0,0.2],'labelpad':35},#
            'r_s':{'j':[0,22],'sd':0.5,'lim':[0,15],'labelpad':13},#
            'open_ratio':{'j':[0,16],'sd':0.008,'lim':[0,0.15],'labelpad':30}
        },
        'Polynomial':{
            'N_max':{'j':[0,20],'sd':1,'lim':[0,1500],'labelpad':40},#
            'epi':{'j':[0,8],'sd':0.01,'lim':[0,0.2],'labelpad':35},#
            'r_s':{'j':[0,12],'sd':0.3,'lim':[0,5],'labelpad':13},#
            'open_ratio':{'j':[10,18],'sd':0.008,'lim':[0,0.15],'labelpad':30}
        }
    }
}

In [ ]:
def get_mean_df(plot_df,N,model,metric):
    mean_dict = {f'{metric}_T':[],f'{metric}_E_mean':[],'std':[]}
    sd = para_dict[N][model][metric]['sd']
    for i in error_list:
        plot_df_ = plot_df[plot_df['std']==i]
        for j in range(*para_dict[N][model][metric]['j']):
            mean_E = np.mean(plot_df_[(plot_df_[f'{metric}_T']<=(j+1)*sd) & (plot_df_[f'{metric}_T']>=j*sd)][f'{metric}_E'])
            mean_dict[f'{metric}_T'].append((j+0.5)*sd)
            mean_dict['std'].append(i)
            mean_dict[f'{metric}_E_mean'].append(mean_E)
    return pd.DataFrame(mean_dict) 

In [ ]:
'''
def draw_by_metric(metric):
    fig = plt.figure(figsize=(30, 30))
    outer = gridspec.GridSpec(3, 3, wspace=0.2, hspace=0.2)

    i=0

    for ax_idx,model in enumerate(model_list):
        ax = fig.add_subplot(3,1,ax_idx+1)
        ax.set_ylabel(f'{model} model',size=30,labelpad=50)
        ax.spines['top'].set_color('none')
        ax.spines['bottom'].set_color('none')
        ax.spines['left'].set_color('none')
        ax.spines['right'].set_color('none')
        ax.tick_params(labelcolor='w', top=False, bottom=False, left=False, right=False)

    for ax_idx,N in enumerate(N_list):
        ax = fig.add_subplot(1,3,ax_idx+1)
        ax.set_xlabel(f'N = {N}',size=30,labelpad=50)
        ax.spines['top'].set_color('none')
        ax.spines['bottom'].set_color('none')
        ax.spines['left'].set_color('none')
        ax.spines['right'].set_color('none')
        ax.tick_params(labelcolor='w', top=False, bottom=False, left=False, right=False)

    for ax_idx in range(1,10):
        N = N_list[ax_idx%3-1]
        model = model_list[(ax_idx-1)//3]
        ax = fig.add_subplot(3,3,ax_idx)
        if ax_idx%3 == 0:
            labelpad = 40
        elif ax_idx%3 == 1:
            labelpad = 15
        elif ax_idx%3 == 2:
            labelpad = 20
        ax.set_ylabel(f'{metric} of measured fitness landscape',size=20,labelpad=para_dict[N][model][metric]['labelpad'])
        ax.set_xlabel(f'{metric} of true fitness landscape',size=20,labelpad=16)
        ax.spines['top'].set_color('none')
        ax.spines['bottom'].set_color('none')
        ax.spines['left'].set_color('none')
        ax.spines['right'].set_color('none')
        ax.tick_params(labelcolor='w', top=False, bottom=False, left=False, right=False)

    for model in model_list:
        for N in N_list:
            inner = gridspec.GridSpecFromSubplotSpec(2, 2,
                            subplot_spec=outer[i], wspace=0.15, hspace=0.15)
            plot_df = plot_dict[N][model][metric]
            mean_df = get_mean_df(plot_df,N,model,metric)
            lim = para_dict[N][model][metric]['lim']
            for j,std in enumerate(error_list):
                ax = plt.Subplot(fig, inner[j])
                ax.scatter(x=f'{metric}_T',y=f'{metric}_E',alpha=0.2, data=plot_df[plot_df['std']==std],label=None,s=10)
                mean_df_ = mean_df[mean_df['std']==std]
                ax.plot(mean_df_[f'{metric}_T'],mean_df_[f'{metric}_E_mean'],'r-',label=f'std {std}')
                ax.plot(lim,lim,'black',linewidth=2)
                if metric == 'open_ratio':
                    #ax.set_xticks([0, 0.05, 0.1, 0.15])
                    ax.set_xticklabels(['0', '0.05', '0.1', '0.15'])
                ax.set_title(f'std = {std}',fontsize=25)
                ax.label_outer()
                ax.set_ylim(lim)
                ax.set_xlim(lim)
                ax.tick_params(labelsize=20)
                
                fig.add_subplot(ax)
            i+=1
    fig.show()
'''

In [ ]:
# Stratified by metrics, Figs not shown in the paper
# draw_by_metric('N_max')

In [ ]:
# Stratified by metrics, Figs not shown in the paper
# draw_by_metric('epi')

In [ ]:
# Stratified by metrics, Figs not shown in the paper
# draw_by_metric('r_s')

In [ ]:
# Stratified by metrics, Figs not shown in the paper
# draw_by_metric('open_ratio')

In [ ]:
ruggedness_measure_dict = {'N_max':'$N_{max}$','epi':'$F_{rse}$','r_s':'$r/s$','open_ratio':'$F_{bp}$'}

In [ ]:
def draw_N(N):
    fig = plt.figure(figsize=(30, 40))
    outer = gridspec.GridSpec(4, 3, wspace=0.2, hspace=0.2)

    i=0

    for ax_idx,model in enumerate(model_list):
        ax = fig.add_subplot(1,3,ax_idx+1)
        ax.set_title(f'{model}',size=40, pad=35)
        ax.spines['top'].set_color('none')
        ax.spines['bottom'].set_color('none')
        ax.spines['left'].set_color('none')
        ax.spines['right'].set_color('none')
        ax.tick_params(labelcolor='w', top=False, bottom=False, left=False, right=False)

    for ax_idx in range(1,13):
        model = model_list[(ax_idx-1)%3]
        metric = metric_list[(ax_idx-1)//3]
        ax = fig.add_subplot(4,3,ax_idx)
        if ax_idx%3 == 0:
            labelpad = 40
        elif ax_idx%3 == 1:
            labelpad = 15
        elif ax_idx%3 == 2:
            labelpad = 20
        if ax_idx%3 == 1:
            ax.set_ylabel(f'Inferred {ruggedness_measure_dict[metric]}',size=25,labelpad=para_dict[N][model][metric]['labelpad'])
            ax.set_xlabel(f'True {ruggedness_measure_dict[metric]}',size=25,labelpad=16)
        ax.spines['top'].set_color('none')
        ax.spines['bottom'].set_color('none')
        ax.spines['left'].set_color('none')
        ax.spines['right'].set_color('none')
        ax.tick_params(labelcolor='w', top=False, bottom=False, left=False, right=False)
    for metric in metric_list:
        for model in model_list:
            inner = gridspec.GridSpecFromSubplotSpec(2, 2,
                            subplot_spec=outer[i], wspace=0.15, hspace=0.15)
            plot_df = plot_dict[N][model][metric]
            mean_df = get_mean_df(plot_df,N,model,metric)
            lim = para_dict[N][model][metric]['lim']
            if metric == 'open_ratio':
                lim = [1-lim[1],1-lim[0]]
                
            for j,std in enumerate(error_list):
                ax = plt.Subplot(fig, inner[j])
                if metric == 'open_ratio':
                    ax.scatter(x=1-plot_df[plot_df['std']==std]['open_ratio_T'],\
                               y=1-plot_df[plot_df['std']==std]['open_ratio_E'],\
                               alpha=0.2,label=None,s=10)
                    mean_df_ = 1 - mean_df[mean_df['std']==std]
                    ax.set_xticks([0.85,0.9,0.95,1])
                    ax.set_xticklabels(['0.85', '0.9', '0.95', '1'])
                else:
                    ax.scatter(x=f'{metric}_T',y=f'{metric}_E',alpha=0.2, data=plot_df[plot_df['std']==std],label=None,s=10)
                    mean_df_ = mean_df[mean_df['std']==std]
                ax.plot(mean_df_[f'{metric}_T'],mean_df_[f'{metric}_E_mean'],'r-',label=f'std {std}')
                ax.plot(lim,lim,'black',linewidth=2)
                
                ax.text(0.02, 0.98, f'sd = {std}', horizontalalignment='left',
                        verticalalignment='top', transform=ax.transAxes, fontsize=25)
                ax.label_outer()
                ax.set_ylim(lim)
                ax.set_xlim(lim)
                ax.tick_params(labelsize=20)
                
                fig.add_subplot(ax)
            i+=1
    fig.show()

In [ ]:
draw_N(10)

In [ ]:
draw_N(5)

In [ ]:
draw_N(15)

In [ ]:
'''
def draw_separate(metric):
    N = 10
    fig = plt.figure(figsize = [25,20])
    
    for ax_idx,model in enumerate(model_list):
        ax = fig.add_subplot(3,1,ax_idx+1)
        ax.spines['top'].set_color('none')
        ax.spines['bottom'].set_color('none')
        ax.spines['left'].set_color('none')
        ax.spines['right'].set_color('none')
        ax.tick_params(labelcolor='w', top=False, bottom=False, left=False, right=False)
        ax.set_ylabel(f'{model}',size=30,labelpad=70)

    for ax_idx,std in enumerate(error_list):
        ax = fig.add_subplot(1,4,ax_idx+1)
        ax.spines['top'].set_color('none')
        ax.spines['bottom'].set_color('none')
        ax.spines['left'].set_color('none')
        ax.spines['right'].set_color('none')
        ax.tick_params(labelcolor='w', top=False, bottom=False, left=False, right=False)
        ax.set_xlabel(f'std = {std}',size=30,labelpad=50)
    
    for row,model in enumerate(model_list):
        for column,std in enumerate(error_list):
            ax = fig.add_subplot(3,4,row*4+column+1)
            plot_df = plot_dict[N][model][metric]
            mean_df = get_mean_df(plot_df,N,model,metric)
            lim = para_dict[N][model][metric]['lim']

            ax.scatter(x=f'{metric}_T',y=f'{metric}_E',alpha=0.2, data=plot_df[plot_df['std']==std],label=None,s=10)
            mean_df_ = mean_df[mean_df['std']==std]
            ax.plot(mean_df_[f'{metric}_T'],mean_df_[f'{metric}_E_mean'],'r-',label=f'std {std}')
            ax.plot(lim,lim,'black',linewidth=2)
            if metric == 'open_ratio':
                ax.set_xticklabels(['0', '0.05', '0.1', '0.15'])
            ax.set_xlabel(f'std = {std}',fontsize=25)
            ax.set_ylabel(model, fontsize=25)
            ax.set_ylabel(f'{metric} (measured)',size=20,labelpad=para_dict[N][model][metric]['labelpad']-25)
            ax.set_xlabel(f'{metric} (ground true)',size=20,labelpad=8)
            ax.label_outer()
            ax.set_ylim(lim)
            ax.set_xlim(lim)
            ax.tick_params(labelsize=20)
            ax
    plt.subplots_adjust(
        wspace=0.1,
        hspace=0.1)
    #fig.tight_layout()
    fig.show()
'''

In [ ]:
# Stratified by metrics, Figs not shown in the paper
#draw_separate("N_max")

In [ ]:
# Stratified by metrics, Figs not shown in the paper
#draw_separate("epi")

In [ ]:
# Stratified by metrics, Figs not shown in the paper
#draw_separate("r_s")

In [ ]:
# Stratified by metrics, Figs not shown in the paper
#draw_separate("open_ratio")